In [1]:
import requests
import re 
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [2]:
# get the data!! 

email = "rasbands@byu.edu"
ua = f"STAT386-class-scraper/1.0 (+{email})"

# get 4 years of data bc each year has top 50 
years = [2024, 2023, 2022, 2021]
all_airports = []

# loop through each year so it will do this data cleaning for all 4 years of data
for year in years:
    url = f"https://en.wikipedia.org/wiki/List_of_busiest_airports_by_passenger_traffic#{year}"
    r = requests.get(url, headers={"User-Agent": ua, "From": email}, timeout=15)
    r.raise_for_status()
    
    tables = pd.read_html(r.text)
    
    # this will make a copy of the table 
    df = tables[0].copy()
    #make a new column called year so we know which year
    df["Year"] = year
    # append that dataframe to the list of all airports!
    all_airports.append(df)


#  combine all 4 years
airports = pd.concat(all_airports, ignore_index=True)

airports.head() 

/var/folders/2n/z5c1xg815rnc8_7vb9lnwrj40000gn/T/ipykernel_5661/192153501.py:16: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(r.text)
/var/folders/2n/z5c1xg815rnc8_7vb9lnwrj40000gn/T/ipykernel_5661/192153501.py:16: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(r.text)
/var/folders/2n/z5c1xg815rnc8_7vb9lnwrj40000gn/T/ipykernel_5661/192153501.py:16: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(r.text)
/var/folders/2n/z5c1xg815rnc8_7vb9lnwrj40000gn/T/ipykernel_5661/192153501.py:16: FutureWarning: Passing literal html to 'read_html' is deprecated and will

,Rank,Airport,Location,Country,Code (IATA/ICAO),Total passengers,Rank change,% change,Year
0,1.0,Hartsfield–Jackson Atlanta International Airport,"Atlanta, Georgia",United States,ATL/KATL,"108,067,766[4]",NaN,03.3%,2024
1,2.0,Dubai International Airport,"Garhoud, Dubai, Dubai",United Arab Emirates,DXB/OMDB,"092,300,000[5]",NaN,06.2%,2024
2,3.0,Dallas Fort Worth International Airport,"Dallas–Fort Worth, Texas",United States,DFW/KDFW,"087,817,864[6]",NaN,07.4%,2024
3,4.0,Tokyo Haneda Airport,"Ōta, Tokyo",Japan,HND/RJTT,"085,000,000[7]",1.0,08.0%,2024
4,5.0,Heathrow Airport,"Hillingdon, London",United Kingdom,LHR/EGLL,"083,882,140[8]",1.0,05.9%,2024


In [3]:
airports.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Rank              200 non-null    float64
 1   Airport           200 non-null    object 
 2   Location          200 non-null    object 
 3   Country           200 non-null    object 
 4   Code (IATA/ICAO)  200 non-null    object 
 5   Total passengers  200 non-null    object 
 6   Rank change       164 non-null    float64
 7   % change          200 non-null    object 
 8   Year              200 non-null    int64  
dtypes: float64(2), int64(1), object(6)
memory usage: 14.2+ KB


In [6]:

# clean the columns if there are extra spaces or lines just incase
airports.columns = [c.strip().replace("\n", " ") for c in airports.columns]

# change column names
airports.columns = [
    "Rank", "Airport", "Location", "Country", "Code", 
    "TotalPassengers", "RankChange", "PercentChange", "Year"
]

# make the passengers column numeric (it is string)
airports["TotalPassengers"] = (
    airports["TotalPassengers"]
    .astype(str)
    .str.replace(",", "", regex=True) # remove commas
    .str.extract("(\d+\.?\d*)")[0] # get only numbers 
    .astype(float)
)

# convert % change to numeric
airports["Percent change numeric"] = (
    airports["PercentChange"]
    .astype(str)
    .str.extract(r'([-+]?\d+\.?\d*)')     # extract numeric part
    [0]
    .astype(float)
)

airports.head() 

<>:15: SyntaxWarning: invalid escape sequence '\d'
<>:15: SyntaxWarning: invalid escape sequence '\d'
/var/folders/2n/z5c1xg815rnc8_7vb9lnwrj40000gn/T/ipykernel_5661/503298741.py:15: SyntaxWarning: invalid escape sequence '\d'
  .str.extract("(\d+\.?\d*)")[0] # get only numbers


,Rank,Airport,Location,Country,Code,TotalPassengers,RankChange,PercentChange,Year,Percent change numeric
0,1.0,Hartsfield–Jackson Atlanta International Airport,"Atlanta, Georgia",United States,ATL/KATL,108067766.0,NaN,03.3%,2024,3.3
1,2.0,Dubai International Airport,"Garhoud, Dubai, Dubai",United Arab Emirates,DXB/OMDB,92300000.0,NaN,06.2%,2024,6.2
2,3.0,Dallas Fort Worth International Airport,"Dallas–Fort Worth, Texas",United States,DFW/KDFW,87817864.0,NaN,07.4%,2024,7.4
3,4.0,Tokyo Haneda Airport,"Ōta, Tokyo",Japan,HND/RJTT,85000000.0,1.0,08.0%,2024,8.0
4,5.0,Heathrow Airport,"Hillingdon, London",United Kingdom,LHR/EGLL,83882140.0,1.0,05.9%,2024,5.9


In [10]:
airports = airports.drop(columns=["PercentChange"])
print(airports.columns)


Index(['Rank', 'Airport', 'Location', 'Country', 'Code', 'TotalPassengers',
       'RankChange', 'Year', 'Percent change numeric'],
      dtype='object')


In [ ]:
# what is rankchange column? tells you how far it moved up or down in the rankings!


# also the percent change has a green arrow or a red arrow meaning it went up or down...

